In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import get_test_data
from fenics import *
from mshr import *
import numpy as np
import os

In [ ]:
os.chdir('/home/erick/Desktop/FEniCS/FEniCS-projects/diffution systems/predator-prey/regular_alpha_064')

In [ ]:
def plot_field_and_fft(field, field_type, fft_field, t): 
    field=np.loadtxt(field,float)
    fft_field=np.loadtxt(fft_field,float)
    field[field == 0.0] = np.nan
    plt.figure(figsize=(10, 3))
    plt.subplot(1, 2, 1)
    if field_type == 'P':
        plt.imshow(field, cmap = 'seismic' , interpolation = 'bilinear')
        plt.title("$P(t={})$".format(t))
    if field_type == 'N':
        plt.imshow(field, cmap = 'gray' , interpolation = 'bilinear')
        plt.title("$N(t={})$".format(t))
    plt.colorbar() 
    
    plt.subplot(1, 2, 2)
    plt.imshow(fft_field, cmap = 'seismic' , interpolation = 'bilinear')
    if field_type == 'P':
            plt.title("$P_{fft}(t=%s)$" % t)
    if field_type == 'N':
            plt.title("$N_{fft}(t=%s)$" % t)
    plt.colorbar()
    
    plt.tight_layout(pad=-3.0)
#    if field_type == 'P':
#        plt.savefig("fields_P_{}.png".format(t))
#    if field_type == 'N':
#        plt.savefig("fields_N_{}.png".format(t))
    plt.show()

In [ ]:
def dft_mean(field, end_step, end_blocks):
    step=0
    while step <= end_step:
        final_file = 0.0
        for nb in range(1, end_blocks+1):
            file_name = "matrix_" + field +  "_dft_" + str(step) + "_nb_" + str(nb) + ".txt"
            file_load = np.loadtxt(file_name, float)
            final_file += file_load
        final_file = final_file/float(end_blocks)
        #print("step=",step)
        
        f_save = "FFT_" + field + "_" + str(step) + ".txt"
        np.savetxt(f_save, final_file, delimiter="\t")
        step += 5
    plt.imshow(final_file, cmap=plt.cm.seismic)
    plt.colorbar()
    plt.show()

In [ ]:
# Time parameters
dt = 5 # step
T = 2000 # end step
nb = 5

In [ ]:
def power_spectrum(field_type, t, block):
    field_file = "matrix_"  + field_type +"_"+ str(t) + "_" + "nb_" + str(block) + ".txt"
    field=np.loadtxt(field_file,float)
    fft_field = np.fft.fft2(field)
    fft_field=np.fft.fftshift(fft_field)
    fft_field=(np.abs(fft_field))
    max_fft = np.where(fft_field == fft_field.max())
    fft_field[max_fft[0][0],max_fft[1][0]] = 0
    dft_file = "matrix" + "_{}_dft_".format(field_type) + str(t) + "_" + "nb_" + str(block) + ".txt"
    np.savetxt(dft_file, fft_field, delimiter="\t")
    return field_file, dft_file

In [ ]:
for block in range(1,nb+1):
    t = 0  # zero time
    while t <= T:
        field_N, dft_field_N = power_spectrum('N', t, block)
        field_P, dft_field_P = power_spectrum('P', t, block)
        t += dt
plot_field_and_fft(field_N, 'N', dft_field_N, t)
plot_field_and_fft(field_P, 'P', dft_field_P, t)

In [ ]:
dft_mean('N', T, nb)
dft_mean('P', T, nb)

In [ ]:
t = 0  # zero time
while t <= T:
    #print(t)
    corr_nn, corr_pp, corr_np, test_nn = correlation_2D(t)
    #plot_correlation_matrix(corr_nn, corr_pp, corr_np, t)

    matrix_corr_nn_file = "matrix_corr" + "_nn_" + str(t) + ".txt"
    np.savetxt(matrix_corr_nn_file, corr_nn, delimiter="\t")

    matrix_corr_pp_file = "matrix_corr" + "_pp_" + str(t) + ".txt"
    np.savetxt(matrix_corr_pp_file, corr_pp, delimiter="\t")

    matrix_corr_np_file = "matrix_corr" + "_np_" + str(t) + ".txt"
    np.savetxt(matrix_corr_np_file, corr_np, delimiter="\t")
    
    matrix_corr_np_file = "matrix_corr" + "_test_" + str(t) + ".txt"
    np.savetxt(matrix_corr_np_file, test_nn, delimiter="\t")
    t += dt

In [ ]:
t = 15  # zero time
while t < T:
    polar_mean(t, "nn")
    t += dt
plot_correlations_3d('nn', T)

In [ ]:
t = 15  # zero time
while t < T:
    polar_mean(t, "test")
    t += dt
plot_correlations_3d('test', T)

In [ ]:
t = 15  # zero time
while t < T:
    polar_mean(t, "pp")
    t += dt
plot_correlations_3d('pp', T)

In [ ]:
t = 15  # zero time
while t < T:
    polar_mean(t, "np")
    t += dt
plot_correlations_3d('np', T)